# CNN 이진분류(컬러 이미지)
* dataset : 개와 고양이 데이터셋
> kaggle
* 전이학습 진행

In [3]:
!pip install --q ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.5 MB/s eta 0:00:00
time: 547 µs (started: 2024-07-01 08:44:17 +00:00)


In [4]:
# cats_and_dogs 파일 삭제
#!rm -rf cats_and_dogs

time: 247 µs (started: 2024-07-01 08:44:17 +00:00)


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

time: 11.3 s (started: 2024-07-01 08:44:17 +00:00)


## 1. 데이터 준비

In [6]:
# !wget : 대용량 등 다운로드가 안되는 것도 있음
# google drive 대용량 파일 down - gdown이 존재함(코랩에 설치되어 있음) -> !pip install gdown
# zip 파일이 생김 - 쓸 수 있을만큼만

#!gdown https://drive.google.com/uc?id=1dC-Kso5FonYWtbxtJnJ8_dxO_Jlo8weR

time: 372 µs (started: 2024-07-01 08:44:28 +00:00)


In [7]:
#!unzip /content/cats_and_dogs.zip

time: 299 µs (started: 2024-07-01 08:44:28 +00:00)


In [8]:
# hard disk 이미지 데이터 저장
# load해서 사용하려면 폴더 구조를 test, train(아래 cats, dogs)와 같이 만드는 것이 기본

# 루트 지정
data_root = '/content/cats_and_dogs'
train_dir = data_root + '/train'
test_dir = data_root + '/test'

print(train_dir)
print(test_dir)

/content/cats_and_dogs/train
/content/cats_and_dogs/test
time: 901 µs (started: 2024-07-01 08:44:28 +00:00)


### 데이터 로딩

In [9]:
# opencv : 이미지 복잡한 계산
# pillow : 단순한 것
class_name = ['cats', 'dogs']

time: 484 µs (started: 2024-07-01 08:44:28 +00:00)


In [10]:
import os
# 하나하나씩 로딩
train_cats_fnames = os.listdir(train_dir + '/cats')
train_dogs_fnames = os.listdir(train_dir + '/dogs')
print(train_cats_fnames[:5])
print(train_dogs_fnames[:5])

FileNotFoundError: [Errno 2] No such file or directory: '/content/cats_and_dogs/train/cats'

time: 567 ms (started: 2024-07-01 08:44:28 +00:00)


In [ ]:
import PIL

X_train = []
y_train = []

# padding을 채워서 정사각형으로 만들어서 사이즈를 바꾸면 적어도 이미지가 찌글어지지 않음
# 까만색으로 채움
# resizing : 244로 바꿈
for fname in train_cats_fnames:
  image = PIL.Image.open(train_dir + '/cats/' + fname)
  image = image.resize((244, 244)) # 튜플 형태
  arr = np.array(image) # numpy array 로 변경
  X_train.append(arr)
  y_train.append(0) # 정답을 맞춰야 함(고양이는 0)

len(X_train), len(y_train)

In [ ]:
for fname in train_dogs_fnames:
  image = PIL.Image.open(train_dir + '/dogs/' + fname)
  image = image.resize((244, 244)) # 튜플 형태
  arr = np.array(image) # numpy array 로 변경
  X_train.append(arr)
  y_train.append(1) # 정답을 맞춰야 함(강아지는 1)

len(X_train), len(y_train)

In [ ]:
# 앞으로 다루기 위해서 numpy로 변경
X_train = np.array(X_train)
y_train = np.array(y_train)

print(type(X_train), type(y_train))
print(X_train.shape, y_train.shape)

## 2. 학습, 검증 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle = True, random_state = 42)

print(X_train.shape, X_val.shape)
print(y_train.shape, y_val.shape)

In [ ]:
sample = np.random.randint(2080, size = 25)

# sample 25개를 그리기
fig = plt.figure(figsize = (8, 8))
for i, idx in enumerate(sample):
    plt.subplot(5, 5, i+1) # 5*5, 1번부터
    plt.imshow(X_train[idx], cmap = 'gray') # color map
    plt.axis('off') # 축을 지움
    plt.title(y_train[idx]) # 정답값 - 이름으로 출력
fig.tight_layout() # 보기 좋게 변경
plt.show()

## 3. 전처리

In [ ]:
X_train_s = X_train.astype('float') / 255.0
X_val_s = X_val.astype('float') / 255.0

## 4. 모델
* 기존에 있는 모델 가져오기
* 기존에 유명한 모델은 라이브러리에서 제공을 함 - 미리 학습을 다해서 가중치까지 넣은 상태로 제공
* 다운로드 받아서 데이터를 넣으면 분류하지만, 우리는 원하는 형태로 바꿔서 사용

In [ ]:
from keras.applications import VGG16
# imgenet으로 학습된 상태로 줌, include_top : 이미지 특성을 받는 쪽만 가져오기(False)
base_model = VGG16(weights = 'imagenet',
                   include_top = False,
                   input_shape = (244, 244, 3))

base_model.trainable = False # 또 학습을 시킬 것인지? False : 기존 그대로 사용
base_model.summary()

In [ ]:
# flatten하고 분류기는 우리가 생성
# 학습 total parameter가 1400만개
from keras import layers

model = keras.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation = 'sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras import callbacks

es = callbacks.EarlyStopping(monitor='val_loss', patience=5)
# 갱신된 값을 파일로 저장할 수 있음 - 각 epochs마다 or 기타 등 옵션을 줄 수 있음
ckpt_path = './temp/cats_dogs.ckpt'
ckpt = keras.callbacks.ModelCheckpoint(
    ckpt_path,
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True # 가중치만 저장할지 - False: layer + 가중치 정보
    )

In [ ]:
EPOCHS = 10
BATCH_SIZE = 32

history = model.fit(X_train_s, y_train,
                    epochs = EPOCHS,
                    batch_size = BATCH_SIZE,
                    callbacks = [es, ckpt],
                    validation_data = (X_val_s, y_val))

In [ ]:
def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.plot(hist['epoch'], hist['loss'], label='train loss')
    plt.plot(hist['epoch'], hist['val_loss'], label='val loss')
    plt.title('Loss Curve')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.plot(hist['epoch'], hist['accuracy'], label='train accuracy')
    plt.plot(hist['epoch'], hist['val_accuracy'], label='val accuracy')
    plt.title('Accuracy Curve')
    plt.legend()
    plt.show()

In [ ]:
plot_history(history)

In [ ]:
# 이 모델이 베스트인 상태임
# 베스트 체크포인트 로딩...
model.load_weights(ckpt_path)

## 5. 평가

In [ ]:
# test 돌면서 읽어오면 됨
import os
# 하나하나씩 로딩
test_cats_fnames = os.listdir(test_dir + '/cats')
test_dogs_fnames = os.listdir(test_dir + '/dogs')
print(test_cats_fnames[:5])
print(test_dogs_fnames[:5])

In [ ]:
# test 데이터 전처리
import PIL

X_test = []
y_test = []

# padding을 채워서 정사각형으로 만들어서 사이즈를 바꾸면 적어도 이미지가 찌글어지지 않음
# 까만색으로 채움
# resizing : 244로 바꿈
for fname in test_cats_fnames:
  image = PIL.Image.open(test_dir + '/cats/' + fname)
  image = image.resize((244, 244)) # 튜플 형태
  arr = np.array(image) # numpy array 로 변경
  X_test.append(arr)
  y_test.append(0) # 정답을 맞춰야 함(고양이는 0)

len(X_test), len(y_test)

In [ ]:
for fname in test_dogs_fnames:
  image = PIL.Image.open(test_dir + '/dogs/' + fname)
  image = image.resize((244, 244)) # 튜플 형태
  arr = np.array(image) # numpy array 로 변경
  X_test.append(arr)
  y_test.append(1) # 정답을 맞춰야 함(고양이는 0)

len(X_test), len(y_test)

In [ ]:
X_test = np.array(X_test)
y_test = np.array(y_test)

print(type(X_test), type(y_test))
print(X_test.shape, y_test.shape)

In [ ]:
X_test_s = X_test.astype('float') / 255.0
print(X_test_s, X_test_s.shape)

In [ ]:
y_pred = model.predict(X_test_s)
y_pred = y_pred.flatten()
y_pred = np.where(y_pred > 0.5, 1, 0)
y_pred

In [ ]:
y_test

In [ ]:
y_test.shape, y_pred.shape

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
def print_metrics(y_true, y_pred, aver='macro'):
    print("accuracy:", accuracy_score(y_true, y_pred))
    print("recall_score:", recall_score(y_true, y_pred, average = aver))
    print("precision_score:", precision_score(y_true, y_pred, average = aver))
    print("f1_score:", f1_score(y_true, y_pred, average = aver))
    cfm = confusion_matrix(y_true, y_pred)
    s = sns.heatmap(cfm, annot=True, cmap='Blues', fmt = 'd', cbar=False)
    s.set(xlabel='Prediction', ylabel='Actual')
    plt.show()
print_metrics(y_test, y_pred, aver='macro')

In [ ]:
wrong_data = np.where(y_test != y_pred)[0]
wrong_len = pd.Series(wrong_data).value_counts().sum()
print(wrong_len)

In [ ]:
wrong_data = np.random.randint(34, size = 25)

fig = plt.figure(figsize = (12, 12))
for i, idx in enumerate(wrong_data):
    plt.subplot(5, 5, i+1) # 5*5, 1번부터
    plt.imshow(X_test[idx], cmap = 'gray') # color map
    plt.axis('off') # 축을 지움
    plt.title(f'pred:{class_name[y_pred[idx]]}({class_name[y_test[idx]]})') # 정답값
fig.tight_layout() # 보기 좋게 변경
plt.show()